# LMSys Chatbot Arena

Propose, document and defend a solution that can determine which answer is better, or if there is a tie, based on criteria you define..  


**Proposal**: Boosting is a machine learning technique that combines the results of several weak model trainings to create a strong model. The process uses a sequence of interactions, where weights are used for the errors obtained in each training session.

**Example**: XGBoost is a machine learning algorithm based on gradient boosting and using decision trees, where each tree tries to correct the errors of the previous tree.

**Features**: For this problem, we will use features extracted from the texts in our dataset.

In this case, we can use exploratory analysis to establish features add to our classification, such as:
- Size of the texts;
- Words present in the question and answears;
- Difference between answears, etc.

## 1 - Pre-processing

In [ ]:
# Librarys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 14,5

import seaborn as sns
sns.set_style = 'whitegrid'

import plotly.express as px
px.defaults.template = "plotly_dark"

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Classification models
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')

import re
import string

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# loading datasets
df_train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df_test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
df_submission = df_test.copy()
sample_example = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
df_train.head()

In [ ]:
# Sample example visualization
sample_example.head()

In [ ]:
# train dataset
df_test.head()

In [ ]:
# Dataset's shape
print(f'Train dataset shape: {df_train.shape}')
print(f'Test dataset shape: {df_test.shape}')

In [ ]:
# Dataset info
df_train.info()

In [ ]:
# Null values train
df_train.isnull().sum()

In [ ]:
# Null values test
df_test.isnull().sum()

In [ ]:
# Text Preprocessing 
def preprocess(string):
    strip = string.strip('[]')
    parts = [x.strip('"') for x in strip.split('","')]
    return ''.join(parts)
columns = ['prompt','response_a','response_b']

for colum in df_train[columns]:
    df_train[colum] = df_train[colum].apply(preprocess)
    
for colum in df_test[columns]:
    df_test[colum] = df_test[colum].apply(preprocess)
    
df_train.head(2)

In [ ]:
# create new columns for data exploration
columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

def class_label(df):
    df['class_label'] = None
    
    for index, row in df.iterrows():
        for col in columns:
            if row[col] == 1:
                df.at[index, 'class_label'] = col
                break

# Call the function on your DataFrame
class_label(df_train)

In [ ]:
# Define class number
class_number = {'winner_model_a':0 ,'winner_model_b':1,'winner_tie':2}

df_train['class'] = df_train['class_label'].map(class_number)
df_train.head(2)

In [ ]:
# Which model was chosen
def chose_model(row):
    if row['class'] == 0:
        return row['model_a']
    elif row['class'] == 1:
        return row['model_b']
    else:
        return 'tie'

df_train['chose_model'] = df_train.apply(chose_model, axis=1)
df_train.head(2)

## 2 - Data Exploration

In [ ]:
df_train[['winner_model_a','winner_model_b','winner_tie']].mean()

In [ ]:
# Analysing class distribuitions
# Objective: is the base balanced?
fig = px.pie(df_train[['winner_model_a','winner_model_b','winner_tie']].mean(), names=df_train['class_label'], title='Class distribuition')
fig.update_layout(width=600, height=400)
fig.show()

In [ ]:
# Models distribuitions
# Objective: The winningest models in the comparison
df_models = df_train.groupby('chose_model')['class'].count().reset_index().sort_values(by='class',ascending=False)
df_models.rename(columns={'chose_model':'model'}, inplace=True)

# Results Graphs
fig = px.bar(df_models[1:11].sort_values(by='class'), y='model', x='class', title='Top 10 Most Chosen Models', text_auto='.3s')
fig.update_layout(width=600, height=400)
fig.show()

In [ ]:
# Models chosen from those mentioned
# Objective: The most of these models
df_uni1 = df_train.groupby('model_a')['id'].count().reset_index().sort_values('id', ascending=False)
df_uni1.rename(columns={'model_a':'model'}, inplace=True)
df_uni2 = df_train.groupby('model_b')['id'].count().reset_index().sort_values('id', ascending=False)
df_uni2.rename(columns={'model_b':'model'}, inplace=True)

df_uni = df_uni1.merge(df_uni2, how='left', on = 'model')
df_uni['total_mentions'] = df_uni['id_x'] + df_uni['id_y']
mentions = df_uni['total_mentions'].sum()
df_uni['%mentions'] = (df_uni['total_mentions'] / mentions) * 100

df_uni = df_uni.merge(df_models, how='left', on = 'model')
df_uni['chose_in_mentions'] = (df_uni['class'] / df_uni['total_mentions']) * 100
df_uni = df_uni.sort_values(by = 'chose_in_mentions', ascending=False)[:10]
df_uni.head(3)

In [ ]:
# Results - Graphs
fig = px.bar(df_uni.sort_values('chose_in_mentions'), y='model', x='chose_in_mentions', title='% Top 10 Most Chosen Models',
             text_auto='.3s')
fig.update_layout(width=600, height=400)
fig.show()

## 3 Modeling

In [ ]:
%%time

df = df_train.copy()

# Word Processing
stopwords_list = stopwords.words('english')

def word_processing(text):
    '''Process text: remove special characters, convert to lowercase, tokenize, and remove stopwords'''
    text = re.sub(r'[/<>()|\+\-\$%&#@\'\"]+', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize the text
    filtered_tokens = [token for token in tokens if token not in stopwords_list]  # Remove stopwords
    return filtered_tokens

# Features engineering
def count_token(tokens):
    '''Count number of tokens'''
    return len(tokens)

def diff_response(tokens1, tokens2):
    '''Difference in token count between responses'''
    len_1 = len(tokens1)
    len_2 = len(tokens2)
    diff = abs(len_1 - len_2)
    return len_1, len_2, diff

def prompt_u_response(text1, text2):
    '''Common words between the prompt and the responses'''
    conj1 = set(text1.split())
    conj2 = set(text2.split())
    intersec = conj1.intersection(conj2)
    return len(intersec)

def aUb(text1, text2):
    '''Common words between the responses'''
    conj1 = set(text1.split())
    conj2 = set(text2.split())
    intersec = conj1.intersection(conj2)
    return len(intersec)

def lexical_diversity(text):
    '''Proportion of unique words'''
    return len(set(text)) / len(text) if text else 0

def avg_words_per_sentence(text):
    '''Calculate average number of words per sentence'''
    sentences = sent_tokenize(text)
    word_count = sum(len(word_tokenize(sentence)) for sentence in sentences)
    return word_count / len(sentences) if sentences else 0

def sentence_diversity(text):
    '''Calculate sentence diversity (variety in sentence length)'''
    sentences = sent_tokenize(text)
    lengths = [len(word_tokenize(sentence)) for sentence in sentences]
    return len(set(lengths)) / len(lengths) if lengths else 0

def Simylarity(text1, text2):
    '''Calculate the similaty between prompt and responses'''
    tokenA = nlp(text1)
    tokenB = nlp(text2)
    return tokenA.similarity(tokenB)

df['similaty_promptUresponse_a'] = df.apply(lambda row: Simylarity(row['prompt'],row['response_a']), axis=1)
df['similaty_promptUresponse_b'] = df.apply(lambda row: Simylarity(row['prompt'],row['response_b']), axis=1)


# Applying word processing to columns
for col in ['prompt', 'response_a', 'response_b']:
    df[col] = df[col].apply(word_processing)
    df[f'{col}_count_token'] = df[col].apply(count_token)

# Applying additional features
for index, row in df.iterrows():
    for col in ['response_a', 'response_b']:
        tokens = row[col]
        text = ' '.join(tokens)
        
        df.at[index, f'{col}_lexical_diversity'] = lexical_diversity(tokens)
        df.at[index, f'{col}_avg_words_per_sentence'] = avg_words_per_sentence(text)
        #df.at[index, f'{col}_keyword_usage'] = keyword_usage(text, keywords)
        df.at[index, f'{col}_sentence_diversity'] = sentence_diversity(text)
        
        # Existing features
        df.at[index, f'{col}_count_token'] = count_token(tokens)

# Applying diferences functions in the train dataframe 
for index, row in df.iterrows():
    response_a_tokens = row['response_a']
    response_b_tokens = row['response_b']
    response_a_text = ' '.join(response_a_tokens)
    response_b_text = ' '.join(response_b_tokens)
    
    len_a, len_b, diff = diff_response(response_a_tokens, response_b_tokens)
    df.at[index, 'response_len_a'] = len_a
    df.at[index, 'response_len_b'] = len_b
    df.at[index, 'response_diff'] = diff
    
    common_words_ab = aUb(response_a_text, response_b_text)
    df.at[index, 'common_words_ab'] = common_words_ab
    
    prompt_tokens = row['prompt']
    prompt_text = ' '.join(prompt_tokens)
    common_words_prompt_a = prompt_u_response(prompt_text, response_a_text)
    common_words_prompt_b = prompt_u_response(prompt_text, response_b_text)
    df.at[index, 'common_words_prompt_a'] = common_words_prompt_a
    df.at[index, 'common_words_prompt_b'] = common_words_prompt_b
    

df.head(2)

In [ ]:
df[['similaty_promptUresponse_a','similaty_promptUresponse_b']].describe().T

In [ ]:
%%time

# features and target selection
X = df.drop(['id','model_a','model_b','prompt','response_a','response_b','winner_model_a','winner_model_b','winner_tie',
            'class_label','chose_model','class'], axis=1).values
y = df['class'].values

# Split train and valid
X_train,X_valid,y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# Models use
models = {
    'XGBoost': XGBClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(random_state=42,verbose=False)
}

# Trainning the models
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Metrics
result_loss = []
result_models = []

for model_name, model in models.items():
    i = 0
    print(f'Model {model_name}')

    model_losses = []  # Loss Models List
    
    for train_index, valid_index in skf.split(X, y):
        X_train_fold, X_test_fold = X[train_index], X[valid_index]
        y_train_fold, y_test_fold = y[train_index], y[valid_index]
        
        # Training the model
        model.fit(X_train_fold, y_train_fold)
        
        # Metrics evaluation
        y_test_pred_proba = model.predict_proba(X_test_fold)
        loss = metrics.log_loss(y_test_fold, y_test_pred_proba)
        model_losses.append(loss)  # Add loss in list
        
        # Display the results
        print(f'Fold {i} | Log Loss: {loss}')
        i += 1
        
        mean_loss = np.mean(model_losses)
        result_loss.append(model_losses)
        result_models.append((model_name, model_losses))

    print(f'Mean Loss for model {model_name} is {mean_loss}')
    print('---' * 30)

In [ ]:
# Plot results
data_dict = {'CV': list(range(0, 5))}

for model, values in result_models:
    data_dict[model] = values

results_df = pd.DataFrame(data_dict)

fig = px.line(results_df, x='CV', y=['XGBoost','GradientBoosting','CatBoost'], title='Loss Models')
fig.update_layout(width=600, height=400)
fig.show()

In [ ]:
%%time
# The best model are GradientBoosting
# Now, we'll use the gridsearch for tunning hyperparametros
model_grad = GradientBoostingClassifier()
params = {
    'n_estimators':[100, 150, 170],
    'max_depth': [1, 3, 5],
    'learning_rate': [0.1, 0.01]
}

grid = GridSearchCV(model_grad, params, n_jobs=-1, cv=5, scoring='neg_log_loss', verbose=False)
grid.fit(X_train, y_train)

In [ ]:
print(f'The best params {grid.best_params_}')

In [ ]:
# Selection the best model
best_model = grid.best_estimator_

In [ ]:
# Avaluation the best model
y_predict_valid_prob = best_model.predict_proba(X_valid)

logloss = metrics.log_loss(y_valid, y_predict_valid_prob)

print(f'LogLoss the Best Models with GridSearch: {logloss}')

There was no significant improvement with GridSearch

In [ ]:
# Features importance
features_importance = best_model.feature_importances_
features_names = ['prompt_count_token', 'response_a_count_token',
       'response_b_count_token', 'response_a_lexical_diversity',
       'response_a_avg_words_per_sentence', 'response_a_sentence_diversity',
       'response_b_lexical_diversity', 'response_b_avg_words_per_sentence',
       'response_b_sentence_diversity', 'response_len_a', 'response_len_b',
       'response_diff', 'common_words_ab', 'common_words_prompt_a',
       'common_words_prompt_b','similaty_promptUresponse_a', 'similaty_promptUresponse_b']

fig = px.bar(y=features_names, x=features_importance, color=features_importance, title='Features Importance')
scale = px.colors.sequential.BuGn
fig.update_traces(marker=dict(colorscale=scale))
fig.update_layout(width=800, height=500)
fig.show()

Above we can see the most important features for the model

In [ ]:
# Preparing the test data to predict
df_test['similaty_promptUresponse_a'] = df_test.apply(lambda row: Simylarity(row['prompt'],row['response_a']), axis=1)
df_test['similaty_promptUresponse_b'] = df_test.apply(lambda row: Simylarity(row['prompt'],row['response_b']), axis=1)



# Applying word processing to columns
for col in ['prompt', 'response_a', 'response_b']:
    df_test[col] = df_test[col].apply(word_processing)
    df_test[f'{col}_count_token'] = df_test[col].apply(count_token)

# Applying additional features
for index, row in df_test.iterrows():
    for col in ['response_a', 'response_b']:
        tokens = row[col]
        text = ' '.join(tokens)
        
        df_test.at[index, f'{col}_lexical_diversity'] = lexical_diversity(tokens)
        df_test.at[index, f'{col}_avg_words_per_sentence'] = avg_words_per_sentence(text)
        #df.at[index, f'{col}_keyword_usage'] = keyword_usage(text, keywords)
        df_test.at[index, f'{col}_sentence_diversity'] = sentence_diversity(text)
        
        # Existing features
        df_test.at[index, f'{col}_count_token'] = count_token(tokens)


# Applying diferences functions in the test dataframe 
for index, row in df_test.iterrows():
    response_a_tokens = row['response_a']
    response_b_tokens = row['response_b']
    response_a_text = ' '.join(response_a_tokens)
    response_b_text = ' '.join(response_b_tokens)
    
    len_a, len_b, diff = diff_response(response_a_tokens, response_b_tokens)
    df_test.at[index, 'response_len_a'] = len_a
    df_test.at[index, 'response_len_b'] = len_b
    df_test.at[index, 'response_diff'] = diff
    
    common_words_ab = aUb(response_a_text, response_b_text)
    df_test.at[index, 'common_words_ab'] = common_words_ab
    
    prompt_tokens = row['prompt']
    prompt_text = ' '.join(prompt_tokens)
    common_words_prompt_a = prompt_u_response(prompt_text, response_a_text)
    common_words_prompt_b = prompt_u_response(prompt_text, response_b_text)
    df_test.at[index, 'common_words_prompt_a'] = common_words_prompt_a
    df_test.at[index, 'common_words_prompt_b'] = common_words_prompt_b

df_test.drop(columns = ['id','prompt','response_a','response_b'], axis=1, inplace=True)
df_test.head()

In [ ]:
# Predict proba in test
y_sub_proba = best_model.predict_proba(df_test)
y_sub_proba

In [ ]:
# Submission df
submission = pd.DataFrame({
    'id':df_submission['id'],
    'winner_model_a': y_sub_proba[:, 0],
    'winner_model_b': y_sub_proba[:, 1],
    'winner_tie': y_sub_proba[:, 2]
})

submission.head()

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)